## Pre-processing des données textuelles :

In [56]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
import string

# Télécharger les ressources nécessaires pour NLTK
nltk.download('stopwords')
nltk.download('punkt')
df = pd.read_csv('movie_review.csv')
stop_words = set(stopwords.words('english'))

def Pre_processing_text(text):
    
    # Convertir le texte en minuscules
    text = text.lower()
    # Tokenization
    tokens = nltk.word_tokenize(text)
    
   # Suppression de la ponctuation et les mots vides
    tokens = [word for word in tokens if word not in string.punctuation]
    tokens = [word for word in tokens if word not in stop_words]
    
    # Rejoindre les mots filtrés en une seule chaîne de texte
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

# Appliquer le prétraitement à chaque texte de la colonne "Text"

df['Preprocessed_Text'] = df['text'].apply(Pre_processing_text)

df['Preprocessed_Text']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0        films adapted comic books plenty success wheth...
1        starters created alan moore eddie campbell bro...
2        say moore campbell thoroughly researched subje...
3        book `` graphic novel `` 500 pages long includ...
4                            words n't dismiss film source
                               ...                        
64715      lack inspiration traced back insipid characters
64716    like many skits current incarnation _saturday_...
64717    watching one `` roxbury `` skits snl come away...
64718                           bump unsuspecting women 's
64719    watching _a_night_at_the_roxbury_ 'll left exa...
Name: Preprocessed_Text, Length: 64720, dtype: object

# Entraînement du modèle Word2Vec :

In [57]:
# from gensim.models import Word2Vec



# # Entraînement du modèle Word2Vec
tokenized_texts = [nltk.word_tokenize(text) for text in df['Preprocessed_Text']]
model = Word2Vec(tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)
print(model)



Word2Vec<vocab=46288, vector_size=100, alpha=0.025>


# Vectorisation des reviews de movies :

In [58]:
def document_vector(doc, model):
    
    doc_embedding = []
    # Pour chaque mot dans le document
    for word in doc.split():
        # Si le mot est dans le vocabulaire du modèle Word2Vec
        if word in model.wv.key_to_index:
            # Ajouter l'embedding du mot à la liste
            doc_embedding.append(model.wv[word])
    # Si le document est vide ou aucun mot n'a été trouvé dans le modèle
    if not doc_embedding:
        return np.zeros(model.vector_size)
    # Calculer la moyenne des embeddings des mots dans le document
    return np.mean(doc_embedding, axis=0)

# Calculer le vecteur de document pour la critique de film
df['Document_Vector'] = df['Preprocessed_Text'].apply(lambda x: document_vector(x, model))


## Division des données :

In [63]:
from sklearn.model_selection import train_test_split

# Séparer les caractéristiques (features) et les étiquettes (labels)
## Convertir les vecteurs de document stockés dans une liste en un DataFrame Pandas
X = pd.DataFrame(df['Document_Vector'].tolist()) 
y = df['tag']  # Les tags sont les étiquettes
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Construction d'un classificateur :

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Création d'une instance de modèle de régression logistique
model = LogisticRegression()

# Entraînement du modèle
model.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
y_pred = model.predict(X_test)




C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Évaluation du modèle :

In [62]:
# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Précision (accuracy) :", accuracy)
print("Rapport de classification :\n", report)

Précision (accuracy) : 0.5618819530284301
Rapport de classification :
               precision    recall  f1-score   support

         neg       0.56      0.46      0.51      6324
         pos       0.56      0.66      0.61      6620

    accuracy                           0.56     12944
   macro avg       0.56      0.56      0.56     12944
weighted avg       0.56      0.56      0.56     12944

